In [1]:
import numpy as np
from scipy.integrate import quad
from scipy import *

Sun model

\begin{align}
\frac{\mathrm{d} S^{0, i}(t)}{S^{0, i}(t)} &=\left(r^{0}-r^{i}\right) \mathrm{~d}t-\left(\mathbf{a}^{i}\right)^{\top} \sqrt{\operatorname{Diag}(\mathbf{v}(t))} \mathrm{~d} \mathbf{W}^v(t)
-\left(\mathbf{b}^{i}\right)^{\top} \left[{\operatorname{Diag}(\mathbf{r}(t))}\right]^\alpha \mathrm{~d} \mathbf{W}^r(t), \quad i=1, \ldots, N \\
\mathrm{d} v_{n}(t)&=\chi_{n}\left(\overline{v_{n}}-v_{n}(t)\right) \mathrm{~d} t+\gamma_{n} \sqrt{v_{n}(t)} \mathrm{~d} Z^v_{n}(t), \quad n=\{1, \ldots, d\}\\
\mathrm{d}  r_{m}(t)&=\lambda_{m}\left(\overline{r_{m}}-r_{m}(t)\right) \mathrm{~d} t+\eta_{m} r_{m}^{\alpha}(t) \mathrm{~d} Z_{m}^{r}(t)), \quad m=\{0,i\}
\end{align}

We need the following numbers first:
\begin{align}
a_{n}^{i, j}=a_{n}^{i}-a_{n}^{j}\\
\tilde{a}_{n}^{i, j}=a_{n}^{i}+a_{n}^{j}\\
\tilde{\rho}_{n, v}=\rho_{n, v}\left(a_{n}^{i}-a_{n}^{j}\right)
\end{align}

In [2]:
param_alpha = 0; #or param_alpha = 0.5
d = 2; # dimention of n, need to be decided


# parameters provided by Guido
a_i = np.array([0.2, 0.8]);
a_j = np.array([0.3, 0.7]);
b_i = 0.4;
b_j = 0.6;


# Data below are from Recchioni_2016
r_bar_i = 0.02;
r_bar_j = 0.00044;
v_bar_1 = 0.05;
v_bar_2 = 0.0345;

chi_1 = 0.3;
chi_2 = 0.65;
gamma_1 = 0.6;
gamma_2 = 0.018;
lambda_i = 0.01;
lambda_j = 3.62;
eta_i = 0.01;
eta_j = 0.0098;



# Contract parameters
T = 1; # maturity
K = 1; # strike price
nsteps = 50000; # monitoring dates
dt = T/nsteps;

# Market parameters
S0 = 1; # spot price
v_1_0 = 0.05;
v_2_0 = 0.089;
r_i_0 = 0.09; # interest rate
r_j_0 = 0.09; # interest rate


# Random numbers
rho_v_1 = -0.3;
rho_v_2 = -0.97;
rho_r_i = -0.23;
rho_r_j = -0.81;



# calculation of the above box
a_i_j = a_i - a_j
tilde_a_i_j = a_i + a_j
tilde_rho_v = rho_v * (a_i_j)


Then we can have
\begin{align}
\mu_{q, v_{n}}=-\frac{1}{2}\left(\chi_{n}+(\imath k-q) \gamma_{n} \tilde{\rho}_{n, v}\right)\\
\varphi_{q}^{v_{n}}(k)=\frac{k^{2}}{2}-\frac{1}{2}\left[\left(q^{2}+q \frac{\tilde{a}_{n}^{i, j}}{a_{n}^{i, j}}\right)-\imath k\left(2 q+\frac{\tilde{a}_{n}^{i, j}}{a_{n}^{i, j}}\right)\right]\\
\zeta_{q, v_{n}}=\frac{1}{2}\left[4 \mu_{q,v_{n}}^{2}+2 \gamma_{n}^{2} \varphi_{q}^{v_{n}}(k)\left(a_{n}^{i, j}\right)^{2}\right]^{1 / 2}\\
\end{align}

In [3]:
# What is k???
k = 1

# parameter /chi  and /gamma for the volatility
chi = np.zeros(d)
gamma = np.zeros(d)

# parameter q why it is 2 and -2
q = 2

# what is the dimension of phi?????
phi_v_q = np.zeros(d)

# calculation of the above box
mu_q_v = -0.5* (chi + (complex(0,1) * k - q)) * gamma * tilde_rho_v
phi_v_q[k] = k**2/2 - 0.5 * ((q**2+ q * tilde_a_i_j[n]/a_i_j[n])- complex(0,1)* k *(2*q + tilde_a_i_j[n]/a_i_j[n]))
zeta_q_v = 0.5 * (4 * mu_q_v * mu_q_v + 2 * gamma* gamma * phi_v_q[k] * a_i_j[n]**2)**0.5

NameError: name 'n' is not defined

Then we have
\begin{align}
s_{q, v_{n}, g} &=1-e^{-2 \zeta_{q, v_{n}} \tau} \\
s_{q, v_{n}, b} &=\left(\zeta_{q, v_{n}}+\mu_{q, v_{n}}\right) e^{-2 \zeta_{q, v_{n}} \tau}+\left(\zeta_{q, v_{n}}-\mu_{q, v_{n}}\right)
\end{align}

In [ ]:
# what is tau in this calculation?????
# T is the strike time
T = 0.0
tau = T- 0

s_q_v_g = 1 - np.exp(-2 * zeta_q_v * tau)
s_q_v_b = (zeta_q_v+ mu_q_v) * np.exp(-2 * zeta_q_v * tau) + zeta_q_v - mu_q_v

\begin{equation}

\underline{W{v, q}^{0(Q)}}\left(t^{\prime}-t, \underline{v}, k ; \underline{\Theta{v}}\right)=e^{-\sum{n=1}^{d}\left(2 \chi{n}^{Q} v{n}^{*(Q)} / \gamma{n}^{2}\right) \ln \left(s{q, v{n}, b} / 2 \zeta{q, v{n}}\right)} e^{-\sum{n=1}^{d}\left(2 \chi{n}^{Q} v{n}^{*(Q)} / \gamma{n}^{2}\right)\left(\mu{q, v{n}}+\zeta{q, v{n}}\right) T} e^{-\sum{n=1}^{d}\left(2 v{n}(0) / \gamma{n}^{2}\right)\left(\zeta{q, v{n}}^{2}-\mu{q, v{n}}^{2}\right) s{q, v{n}, g} / s{q, v_{n}, b}}

\end{equation}

In [ ]:
# volatility
v = np.zeros(T)

# calculation of the above box

# sum the exponential one by one
sum_1 = 0
for n in range(d):
    sum_1 = sum_1 + (2 * chi[n] * v[n] / (gamma[n]**2)) * np.log(s_q_v_b[n] / (2 * zeta_q_v[n]))
sum_2 = 0
for n in range(d):
    sum_2 = sum_2 + (2 * chi[n] * v[n] / (gamma[n]**2)) * (mu_q_v[n]+ zeta_q_v[n]) * T
sum_3 = 0
for n in range(d):
    sum_3 = sum_3 + (2 * v[n] / (gamma[n]**2)) * (zeta_q_v[n]**2 - mu_q_v[n]**2) * s_q_v_g[n] / s_q_v_b[n]
underline_W_q_v = np.exp(-sum_1) * np.exp(-sum_2) * np.exp(-sum_3)

[H]By taking $a^{i, j} \rightarrow 0^{+}$, $\lambda_{m} \rightarrow 0^{+}$, $\eta_{n} \rightarrow 0^{+}$, we can have the call option in the case $q=2$
\begin{equation}
     C_{H}\left(S_{0}^{i, j}, T, E, \underline{r_{0}}, \underline{v_{0}}\right) = e^{-r_{i}(0) T} e^{2 r_{i}(0) T} \frac{S_{0}^{i, j}}{2 \pi} \int^{+\infty}_{-\infty} d k \frac{\left(\frac{S_{0}^{i, j}}{E}\right)^{(1-\imath k)} e^{-\imath k r_{i}(0) T}}{-k^{2}-3 \imath k+2} \underline{W_{v, q}^{0(Q)}}\left(t^{\prime}-t, \underline{v}, k ; \underline{\Theta_{r}}\right)
\end{equation}



In [ ]:
# strike price E, maturity time T
r_i = np.zeros(T)
S_i_j = np.zeros(T)
E = 0.0

call_option_multiplier = np.exp(- r_i[0] * T) * np.exp(2* r_i[0] * T) * S_i_j[0] / (2 * np.pi)
call_option_integrand = d * k * ((S_i_j[0]/ E)**(1-complex(0,1)*k)*np.exp(-complex(0,1)*k*r_i[0]*T)) * underline_W_q_v / (-k**2 -3*k *complex(0,1) + 2)
call_option_integration = quad(call_option_integrand,-np.inf,np.inf)
call_option = call_option_multiplier * call_option_integration

[H]and the put option when $q = -2$
\begin{equation}
     P_{H}\left(S_{0}^{i, j}, T, E, \underline{r}_{0}, \underline{v_{0}}\right) =e^{-r_{i}(0) T} e^{-2 r_{i}(0) T} \frac{S_{0}^{i, j}}{2 \pi} \int_{-\infty}^{+\infty} d k \frac{\left(\frac{S_{0}^{i, j}}{E}\right)^{-(3+\imath k)} e^{-\imath k r_{i}(0) T}}{-k^{2}+5 \imath k+6} \underline{W_{v, q}^{0(Q)}}{\left(t^{\prime}-t, \underline{v}, k ; \underline{\Theta_{r}}\right)}
\end{equation}

In [ ]:
put_option_multiplier = np.exp(- r_i[0] * T) * np.exp(2* r_i[0] * T) * S_i_j[0] / (2 * np.pi)
put_option_integrand = d * k * ((S_i_j[0]/ E)**(-3-complex(0,1)*k)*np.exp(-complex(0,1)*k*r_i[0]*T)) * underline_W_q_v / (-k**2 +5*k *complex(0,1) + 6)
put_option_integration = quad(put_option_integrand,-np.inf, np.inf)
put_option = put_option_multiplier * put_option_integration